#### Managing the model lifecycle in Unity Catalog

#### 01. Programmatically find best run and push model to Unity Catalog for validation

In [0]:
# Paramets 
catalog = "workspace"
db = "customer_churn"
xp_name = "mlops_customer_churn_experiment"
xp_path = f"/Users/samuel.hmariam@shewit.co.uk"


In [0]:
import mlflow


model_name = f"{catalog}.{db}.advanced_mlops_churn"

print(f"Finding best run from {xp_name} and pushing new model version to {model_name}")
mlflow.set_experiment(f"{xp_path}/{xp_name}")

In [0]:
# Let's get our best ml run
best_model = mlflow.search_runs(
  order_by=["metrics.test_f1_score DESC"],
  max_results=1,
  filter_string="status = 'FINISHED' and run_name='mlops_best_run'" #filter on mlops_best_run to always use the notebook 02 to have a more predictable demo
)
# Optional: Load MLflow Experiment as a spark df and see all runs
# df = spark.read.format("mlflow-experiment").load(experiment_id)
best_model

In [0]:
print(f"Registering model to {catalog}.{db}.advanced_mlops_churn")

# Get the run id from the best model
run_id = best_model.iloc[0]['run_id']

# Register best model from experiments run to MLflow model registry
model_details = mlflow.register_model(f"runs:/{run_id}/model", f"{catalog}.{db}.advanced_mlops_churn")

#### 02. Give the registered model a description

In [0]:
from mlflow import MlflowClient


client = MlflowClient()

# The main model description, typically done once.
client.update_registered_model(
  name=model_details.name,
  description="This model predicts whether a customer will churn using the features in the advanced_churn_feature_table table. It is used to power the Telco Churn Dashboard in DB SQL.",
)

In [0]:
# Provide more details on this specific model version
best_score = best_model['metrics.test_f1_score'].values[0]
run_name = best_model['tags.mlflow.runName'].values[0]
version_desc = f"This model version has an F1 validation metric of {round(best_score,4)*100}%. Follow the link to its training run for more details."

client.update_model_version(
  name=model_details.name,
  version=model_details.version,
  description=version_desc
)

# We can also tag the model version with the F1 score for visibility
client.set_model_version_tag(
  name=model_details.name,
  version=model_details.version,
  key="f1_score",
  value=f"{round(best_score,4)}"
)

#### Set the latest model version as the Challenger model

In [0]:
# Set this version as the Challenger model, using its model alias
client.set_registered_model_alias(
  name=f"{catalog}.{db}.advanced_mlops_churn",
  alias="Challenger",
  version=model_details.version
)